Samuel Vilchez: Testing Particle Swarm Optimization Method on the 48 Capitals 

We will be using the Graph, Particle, and PSO classes from tsp_pso https://github.com/marcoscastro/tsp_pso/blob/master/tsp_pso.py to apply the Particle swarm optimization method on the traveling salesman problem. 

In [27]:
from operator import attrgetter
import random, sys, time, copy, math

This is a basic Graph class obtained by tsp_pso which has an extra function from that of a typical graph to obtain a set of randomPaths. The randomness is used to caculate the initial particles for the Particle Swarm Optimization Algorithm.

In [28]:
#This class was obtained by tsp_pso and redefined here
class Graph:
    def __init__(self, amount_vertices):
        self.edges = {} 
        self.vertices = set()
        self.amount_vertices = amount_vertices
        
    def addEdge(self, src, dest, cost = 0):
        if not self.existsEdge(src, dest):
                self.edges[(src, dest)] = cost
                self.vertices.add(src)
                self.vertices.add(dest)
                
    def existsEdge(self, src, dest):
        return (True if (src, dest) in self.edges else False)
    
    def getCostPath(self, path):
        total_cost = 0
        for i in range(self.amount_vertices - 1):
            total_cost += self.edges[(path[i], path[i+1])]
            
        total_cost += self.edges[(path[self.amount_vertices - 1], path[0])]
        return total_cost
    
    def getRandomPaths(self, max_size):
        random_paths, list_vertices = [], list(self.vertices)
        initial_vertices = random.choice(list_vertices)
        if initial_vertices not in list_vertices:
            sys.exit(1)
        
        list_vertices.remove(initial_vertices)
        list_vertices.insert(0, initial_vertices)
        
        for i in range(max_size):
            list_temp = list_vertices[1:]
            random.shuffle(list_temp)
            list_temp.insert(0, initial_vertices)
            if list_temp not in random_paths:
                random_paths.append(list_temp)
                
        return random_paths

In [29]:
#This class was obtained by tsp_pso and redefined here
class Particle:
    def __init__(self, solution, cost):
        self.solution = solution
        self.pbest = solution
        self.cost_current_solution = cost
        self.cost_pbest_solution = cost
        self.velocity = []
    
    def setPBest(self, new_pbest):
        self.pbest = new_pbest
    
    def getPBest(self):
        return self.pbest
    
    def setVelocity(self, new_velocity):
        self.velocity = new_velocity
        
    def getVelocity(self):
        return self.velocity 
    
    def setCurrentSolution(self, solution):
        self.solution = solution
        
    def getCurrentSolution(self):
        return self.solution
    
    def setCostPBest(self, cost):
        self.cost_pbest_solution = cost
        
    def getCostPBest(self):
        return self.cost_pbest_solution
    
    def setCostCurrentSolution(self, cost):
        self.cost_current_solution = cost
        
    def getCostCurrentSolution(self):
        return self.cost_current_solution
    
    def clearVelocity(self):
        del self.velocity[:]

The following class will initialize a fixed sized population with initially random particles (which can be viewed as paths for the graph). The parameters include a graph, the max number of iterations, fixed sized population, and probabilities for the swap operators used to swap elements from a path (particle). It will then set a global best path of the randomized paths. Next, the run function from tsp_pso will update the position and velocity information for each particle at each iteration. Fitness values are evaluated, compared and replaced if better values are obtained.

In [30]:
#This class was obtained by tsp_pso and redefined here
class PSO:
    def __init__(self, graph, iterations, size_population, beta=1, alfa=1):
        self.graph = graph # the graph
        self.iterations = iterations # max of iterations
        self.size_population = size_population # size population
        self.particles = [] # list of particles
        self.beta = beta # the probability that all swap operators in swap sequence (gbest - x(t-1))
        self.alfa = alfa # the probability that all swap operators in swap sequence (pbest - x(t-1))

        # initialized with a group of random particles (solutions)
        solutions = self.graph.getRandomPaths(self.size_population)

        # checks if exists any solution
        if not solutions:
            print('Initial population empty! Try run the algorithm again...')
            sys.exit(1)

        # creates the particles and initialization of swap sequences in all the particles
        for solution in solutions:
            # creates a new particle
            particle = Particle(solution=solution, cost=graph.getCostPath(solution))
            # add the particle
            self.particles.append(particle)

        # updates "size_population"
        self.size_population = len(self.particles)


    # set gbest (best particle of the population)
    def setGBest(self, new_gbest):
        self.gbest = new_gbest

    # returns gbest (best particle of the population)
    def getGBest(self):
        return self.gbest

    def run(self):
        # for each time step (iteration)
        for t in range(self.iterations):
            # updates gbest (best particle of the population)
            self.gbest = min(self.particles, key=attrgetter('cost_pbest_solution'))

            # for each particle in the swarm
            for particle in self.particles:

                particle.clearVelocity() # cleans the speed of the particle
                temp_velocity = []
                solution_gbest = copy.copy(self.gbest.getPBest()) # gets solution of the gbest
                solution_pbest = particle.getPBest()[:] # copy of the pbest solution
                solution_particle = particle.getCurrentSolution()[:] # gets copy of the current solution of the particle

                # generates all swap operators to calculate (pbest - x(t-1))
                for i in range(self.graph.amount_vertices):
                    if solution_particle[i] != solution_pbest[i]:
                        # generates swap operator
                        swap_operator = (i, solution_pbest.index(solution_particle[i]), self.alfa)

                        # append swap operator in the list of velocity
                        temp_velocity.append(swap_operator)

                        # makes the swap
                        aux = solution_pbest[swap_operator[0]]
                        solution_pbest[swap_operator[0]] = solution_pbest[swap_operator[1]]
                        solution_pbest[swap_operator[1]] = aux
                
                # generates all swap operators to calculate (gbest - x(t-1))
                for i in range(self.graph.amount_vertices):
                    if solution_particle[i] != solution_gbest[i]:
                        # generates swap operator
                        swap_operator = (i, solution_gbest.index(solution_particle[i]), self.beta)

                        # append swap operator in the list of velocity
                        temp_velocity.append(swap_operator)

                        # makes the swap
                        aux = solution_gbest[swap_operator[0]]
                        solution_gbest[swap_operator[0]] = solution_gbest[swap_operator[1]]
                        solution_gbest[swap_operator[1]] = aux

                # updates velocity
                particle.setVelocity(temp_velocity)

                # generates new solution for particle
                for swap_operator in temp_velocity:
                    if random.random() <= swap_operator[2]:
                        # makes the swap
                        aux = solution_particle[swap_operator[0]]
                        solution_particle[swap_operator[0]] = solution_particle[swap_operator[1]]
                        solution_particle[swap_operator[1]] = aux
            
                # updates the current solution
                particle.setCurrentSolution(solution_particle)
                # gets cost of the current solution
                cost_current_solution = self.graph.getCostPath(solution_particle)
                # updates the cost of the current solution
                particle.setCostCurrentSolution(cost_current_solution)

                # checks if current solution is pbest solution
                if cost_current_solution < particle.getCostPBest():
                    particle.setPBest(solution_particle)
                    particle.setCostPBest(cost_current_solution)

Here we declare the coordinates for the 48 different capitals. We then create a child class with a parent class of Graph from tsp_pso. We use the distance formula to calculate the weights for all edges.

In [31]:
coordinates = [6734, 1453, 2233, 10, 5530, 1424, 401, 841, 3082, 1644, 7608, 4458, 7573, 3716, 7265, 1268, 6898, 1885, 1112, 2049, 5468, 2606,
5989, 2873, 4706, 2674, 4612, 2035, 6347, 2683, 6107, 669, 7611, 5184, 7462, 3590, 7732, 4723, 5900, 3561, 4483, 3369, 6101, 1110,
5199, 2182, 1633, 2809, 4307, 2322, 675, 1006, 7555, 4819, 7541, 3981, 3177, 756, 7352, 4506, 7545, 2801, 3245, 3305, 6426, 3173,
4608, 1198, 23, 2216, 7248, 3779, 7762, 4595, 7392, 2244, 3484, 2829, 6271, 2135, 4985, 140, 1916, 1569, 7280, 4899, 7509, 3239,
10, 2676, 6807, 2993, 5185, 3258, 3023, 1942];

class USGraph(Graph):
    #The following will generate a graph with every vertex being adjacent to one another
    def create(self):
        for i in range(self.amount_vertices):
            for j in range(self.amount_vertices):
                if i != j: 
                    weight = math.sqrt(pow(coordinates[2*i] - coordinates[2*j],2) + pow(coordinates[2*i+1] - coordinates[2*j+1], 2));
                    self.addEdge(i, j, weight)

We will use the PSO class to run the Particle Swarm Optimization method on the 48 different capitals. We will use 1000 iterations with a population size of 20. 

In [32]:
graph = USGraph(amount_vertices = 48)
graph.create()
psoGraph = PSO(graph, iterations = 1000, size_population = 20, beta = 1, alfa = 1)
start_time = time.time()
psoGraph.run()
elapsed_time = time.time() - start_time
print('gbest: %s \nThe fitness score at this state is: %d\n' % 
      (psoGraph.getGBest().getPBest(), psoGraph.getGBest().getCostPBest()))
print("The elapsed time is: {0:.2f}".format(elapsed_time))

gbest: [34, 38, 23, 4, 47, 28, 25, 3, 44, 9, 27, 26, 29, 6, 8, 42, 18, 35, 41, 7, 46, 20, 40, 14, 22, 33, 10, 43, 31, 13, 0, 36, 16, 12, 19, 11, 17, 39, 5, 30, 21, 24, 45, 32, 37, 15, 2, 1] 
The fitness score at this state is: 101595

The elapsed time is: 2.72


| Method | Time Elapsed(s) | Min Cost Route |
| --- | --- | --- |
| Particle Swarm Method | 2.72 | 101595 |